# Implementing t-SNE in Python with `numpy`

**Authors:** Abdullah AlOthman, Ethan Swartzentruber

**Date:** 4/27/2021

# Abstract

t-SNE, meaning t-distribution symmetric neighbor embedding, is an iterative technique for dimensionality reduction which is mainly focused on making high dimensional data possible to visualize by reducing it to a lower number of dimensions, typically 2 or 3.

TODO

# Background
A recurring problem in machine learning and statistics is the visualization of high dimensional data - it is extremely important to researcher's intuitive understanding that the data be visualized, but straightforward ways of plotting high dimensional data (plotting only a subset of the dimensions, etc) often perform very poorly. Dimensionality reduction is a common way to remedy this problem. Ideally, these techniques are able to collapse the data into a small enough number of dimensions to plot, without losing too much of the structure of the data.

Basic dimensionality redution techniques, such as PCA, are often enough to see some patterns in the data. However, these techniques can be insufficient, or even outright misleading, when applied to datasets where the dimensions have complex, non-linear relationships. The t-SNE algorithm is a more general dimensionality reduction technique, and does not make as many assumptions about the structure of the data. In essence, t-SNE is based on the assumption that datapoints that are close in high dimensional space should be similarly close in low dimensional space, compared to other points. 

It is important to understand a key limitation of the algorithm: t-SNE cannot be applied on new data the way that one can apply PCA, nor can it be reversed. t-SNE is an iterative algorithm, fit by moving around the low dimensional data points until they "match" the high dimenional ones in the particular mathematical sense that t-SNE cares about - it does not yield any notion of what the reduced components are, and as such, cannot be applied on new data that wasn't a part of the initial iterative process.

In general, t-SNE does well when the goal is to visualize data for the researcher's own intuitive understanding of the structure, particularly natural strata of the data. For example, we will show later in this paper that t-SNE can effectively reveal the clustered nature of the MNIST dataset. However, t-SNE is not particularly well-suited for dimensionality reduction preceding the fitting of a model. This is because it cannot be applied on new data that the model predicts. Additionally, because it does not yield a notion of what the components are, t-SNE is not as well suited for feature engineering as other dimensionality reduction techniques.

# Algorithm Description

t-SNE is based on the intuition that if data points are relatively close together in high dimensional space, they should also be relatively close in low dimensional space. In particular, t-SNE uses a notion of "affinity" between data points, which can be interpreted as a likelihood that two points are neighbors. The closer the points are, the higher this affinity, increasing in a non-linear way. The t-SNE algorithm takes place in two phases:

1. Compute the pairwise affinities of the high dimensional points
2. After randomly initializing them, move the low dimensional points around until the low dimensional points affinities are similar to the high dimensional ones. Essentially, points that are too close compared to their distance in high dimensions move away from each other, and points that are too far away move closer.

To gain some basic intuition on the process, we have created an animation of the t-SNE fitting process that shows the low dimensional points being moved around: [TODO LINK]

The full version of the t-SNE algorithm follows. For all of these computations:
* $X$ is the dataset, with $x_i$ being some point $i$
* $Y$ is the dataset's low dimensional embedding, with $y_i$ being some embedded point $i$ corresponding to $x_i$

### 1: Binary search and high dimensional affinities

First, we must compute pairwise affinities in high dimensional space. The user gives a value called "perplexity", which is a measure of how many points each point should consider to be its neighbors - the higher the perplexity, the slower the drop off in affinity based on distance between points. Note that this affinity is asymetric, as a particular point may choose another point as its neighbor, but that point may have much closer points. The perplexity is defined as:

$$Perp(P_i) = 2^{H(P_i)}$$

where $P_i$ is a probability distribution of how likely point $i$ is to pick any other point as its neighbor, and $H(P_i)$ is the shannon entropy of $P_i$, defined as:

$$H(P_i) = - \sum_j{p_{j|i}log_2p_{j|i}}$$

The distribution $p_{j|i}$ is based on the pairwise distances between points $i$ and $j$, and is calculated as follows:

$$p_{j|i} = \frac{exp(-\lVert x_i - x_j\rVert^2/2\sigma_i^2)}{\sum_{i \neq k}{exp(-\lVert x_i - x_k\rVert^2/2\sigma_i^2)}}$$

Where $x_i$, $x_j$ and $x_k$ are data points, and $\sigma_i$ is the standard deviation of a probability distribution centered at x, which defines its affinity to the other datapoints. At the beginning of the algorithm, the $\sigma_i$ values are unknown - the first step of t-SNE is to perform a binary search across the space of all possible $\sigma_i$ values, choosing a sigma for each point that gives its affinity distribution the user's desired perplexity.

### 2: Iterative fitting

t-SNE is an iterative algorithm, fit via gradient descent. The core idea of t-SNE is that the affinity distribution should be similar in high and low dimensional space, and as such, the cost function is defined as the sum of KL-Divergences across the low and high affinity distributions of every point:

$$C = \sum_i{KL(P_i||Q_i)} = \sum_i{\sum_j{p_{j|i}log{\frac{p_{j|i}}{q_{j|i}}}}}$$

Here, $P_i$ is the distribution of affinities in high dimensional space for point $i$, and $Q_i$ is the distribution of lower dimensional affinities by the same point $i$. $Q$ is defined similarly to $P$, however, it is symmetric:

$$q_{ij} = \frac{exp(-\lVert y_i - y_j \rVert^2)}{\sum_{k \neq l}{exp(- \lVert y_k - y_l \rVert^2)}}$$

We found this notation slightly unclear - note that $k$ and $l$ in the denominator are simply two indices - the sum is across all pairwise square norm differences where the two points are not the same.

After computing q_{ij}, we can compute the gradient of the cost function and fit the algorithm:

$$\frac{\delta C}{\delta y_i} = 4 \sum_j{(p_{ij} - q_{ij})(y_i - y_j)}$$

Note that this uses $p_{ij}$, a symmetric version of the high dimensional pairwise affinity distribution that we computed in the preceding section. These values are defined as $p_{ij} = \frac{p_{j|i} + p_{i|j}}{2n}$, where n is the number of data points.

After this calculation, we are ready to update $Y$ as follows:

$$Y^{(t)} = Y^{(t-1)} + \eta \frac{\delta C}{\delta Y} + \alpha^{(t)}(Y^{(t-1)} - Y^{(t-2)})$$

Where $\eta$ and $\alpha$ are the learning rate and momentum coefficient, respectively.

We run this iterative loop for some set number of iterations, or until the cost function is mimized.

# Performance Optimization

### Vectorization

Our primary method of optimization was to create vectorized versions of all the subcomponents of t-SNE. This version vastly outperformed our original version, due to the massive parallelization of vector operations. We were able to achieve over 100x speedup through vectorization, shown in the plots below.

### JIT compilation with `numba`

In addition to vectorization, we applied just-in-time compilation with numba in an attempt to speed things up. However, we saw no increase in performance. We attribute this to numpy being mostly compiled C code anyway - the bulk of time in our algorithm is spent running computations of the gradient, which is almost completely done with numpy matrix operations.


### Other multiprocessing
In addition to parallelization within numpy matrix operations, we considered further parallelization, as the binary searches in the beginning of the algorithm are independent and could be done in parallel. For larger datasets the overhead of movement across hosts may be preferable to increased processing time, but we chose not to implement this technique given that we were running the algorithm on a single machine - the parallelism of numpy already made full use of the CPU's cores, thus the algorithm could not be further parallelized.


### Performance vs. Size of Dataset
![Iterative t-SNE performance](iterative_tsne_perf.png)
![Vectorized t-SNE performance](vector_tsne_perf.png)

# Performance on Data
* Two main applications are understanding shape and seeing clusters.
* Simulated swiss roll is example of the first
* MNIST is the second
* Note that it is difficult to know a "right answer", it is mostly intuitive about viz. However, the reuslts seem sensical.

## Simulated Data: `sklearn.datasets.make_swiss_roll`
* Show time plot based on datapoints included
* Show 2d representation (should be a line)


## Real Data: 8x8 MNIST
* show time plot based on datapoints included
* show 2d representation

# Comparitive Analysis
* Note that this is highly dependent on the data, in particular if the data is linearly separable then PCA or some other, much faster technique can do just as well as t-SNE in a much more intuitive and useful way.
* Show plots for MNIST and swiss roll

# Discussion/Conclusion
* Great tool for viz, not amazing for dim reduction as part of an ML pipeline.

* Init via autoencoder, isntead of via PCA. Still fast, and its nonlinear so might be really good. Might totally obselete t-SNE altogether, though. Might be good for images.

* Fit regression of high dim features on the low ones, to figure out whether specific high dim features control the splitting.

# References
* Original paper
* Van der maaten implementation
* Liam article
* Were the others? Ask Abdullah